# go straight game
- its a 1-D (one dimension) problem with target T and ur status O on the line
- Morvan tutorial ([link](https://morvanzhou.github.io/tutorials/machine-learning/reinforcement-learning/2-1-general-rl/))

In [1]:
import numpy as np
import pandas as pd
import time

# setting env.

In [2]:
# component for Q-table
ACTIONS = ['left','right']
nSTATE = 10 # status 0-9
data = [[0]*len(ACTIONS)]*nSTATE # init. table data with 0
data

[[0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0]]

# setting Q-table

In [3]:
Q_table = pd.DataFrame(data=data, columns=ACTIONS)
Q_table

,left,right
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


# setting Action

In [4]:
EPSILON = 0.8  # 80% choose highest reward action and 20% choose randomly

def choose_action(state):
    state_actions = Q_table.iloc[state]
    # 隨機值大於0.8(epsilon value) 或 第一次進入該state --> 隨機選取action
    if (np.random.uniform() > EPSILON) or (state_actions.all()==0):
        return np.random.choice(ACTIONS) # 隨機選取action
    else:
        return state_actions.argmax() #derive highest reward

# setting feedback from env.

In [5]:
# With state and action you take, you can get env. feedback
def env_feedback(state, action):
    if action == 'right':
        if state == nSTATE-2: #before terminate
            next_state = 'terminate'
            reward = 1
        else:
            next_state = state +1
            reward = 0
    else: # move left
        reward = 0 #this action wont reach target, so reward always = 0 in the action
        if state == 0:
            next_state = state # reah the wall :3
        else:
            next_state = state - 1
    return next_state, reward

# env and agent state visualization

In [6]:
STEP_TIME = 0.1

def update_visual_env(state, episode, step_count):
    env_list = ['-']*(nSTATE-1)+['T']  # '---------T' is our environment
    if state == 'terminate':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_count) # string
        print('\r{}'.format(interaction), end='\n') # wont change line if end='\n'
        time.sleep(2) # sleep 2 sec
        print('\r                      ', end='') # refresh line
    else : 
        env_list[state] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(STEP_TIME)

# Main Q-Learning Process

In [7]:
MAX_EPISODES = 13
GAMMA = 0.9 # decay rate
ALPHA = 0.1 # learning rate

# No need to think but follow Q-Learning algorithm
def Q_Learning():
    for episode in range(MAX_EPISODES):
        step_count = 0
        state = 0
        is_terminated = False
        update_visual_env(state, episode, step_count) # update env and show
        while not is_terminated:
            action = choose_action(state)
            next_state, reward = env_feedback(state, action) #in this case: {terminal:1,otherwise:0}
            #print('state : '+ str(state))
            q_predict = Q_table.ix[state, action] #get Q_table value
            if next_state == 'terminate':
                q_target = reward        # reward=1 ,
                is_terminated = True   # also means episode finish
            else:
                # pick max reward at next state's actions,
                # multiply by decay rate GAMMA, 
                # and Add reward and previous result together 
                q_target = reward + GAMMA * Q_table.iloc[next_state, :].max()
            
            # update Q-table with q_target and predict one,
            # which q_target was measured by next state
            Q_table.ix[state, action] += ALPHA * (q_target - q_predict)
            state = next_state # finally update state
            
            #Optional :P (just for visualization)
            update_visual_env(state, episode, step_count+1) # update env. after state update
            step_count +=1

In [8]:
Q_Learning()
Q_table

o--------T

/Users/bl515/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  app.launch_new_instance()


Episode 1: total_steps = 58
Episode 2: total_steps = 37
Episode 3: total_steps = 141
Episode 4: total_steps = 15
Episode 5: total_steps = 32
Episode 6: total_steps = 10
Episode 7: total_steps = 12
Episode 8: total_steps = 9
Episode 9: total_steps = 11
Episode 10: total_steps = 11
Episode 11: total_steps = 9
Episode 12: total_steps = 12
Episode 13: total_steps = 11
                      

,left,right
0,2.416758e-08,0.000001
1,1.141639e-08,0.000014
2,8.178877e-10,0.000135
3,5.856714e-06,0.001244
4,8.653447e-05,0.006352
5,5.904900e-07,0.033230
6,4.139991e-05,0.128973
7,7.290000e-05,0.364226
8,1.841273e-02,0.745813
9,0.000000e+00,0.000000
